### 3-3．모델작성
#### 데이터의 리드와 확인

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Pandas의 로드
import pandas as pd

# 파일리드
bank_df_new = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/머신러닝 데이터 전처리 입문_로드북/chap3-4/data/bank-prep2.csv', sep=',')

# 선두에서 5행까지 표시
bank_df_new.head()

,age,default,balance,housing,loan,day,duration,campaign,pdays,previous,...,4Q,early,late,midle,long,short,one-more,zero,less,more
0,58,0,2143,1,0,5,261,1,-1,0,...,0,1,0,0,0,1,0,1,1,0
1,36,0,265,1,1,5,348,1,-1,0,...,0,1,0,0,1,0,0,1,1,0
2,25,0,-7,1,0,5,365,1,-1,0,...,0,1,0,0,1,0,0,1,1,0
3,53,0,-3,0,0,5,1666,1,-1,0,...,0,1,0,0,1,0,0,1,1,0
4,24,0,-103,1,1,5,145,1,-1,0,...,0,1,0,0,0,1,0,1,1,0


#### 불균형 데이터의 균형화

In [3]:
!pip show imbalanced-learn

Name: imbalanced-learn
Version: 0.10.1
Summary: Toolbox for imbalanced dataset in machine learning.
Home-page: https://github.com/scikit-learn-contrib/imbalanced-learn
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.9/dist-packages
Requires: joblib, numpy, scikit-learn, scipy, threadpoolctl
Required-by: 


In [4]:
!pip list

Package                       Version
----------------------------- --------------------
absl-py                       1.4.0
alabaster                     0.7.13
albumentations                1.2.1
altair                        4.2.2
anyio                         3.6.2
appdirs                       1.4.4
argon2-cffi                   21.3.0
argon2-cffi-bindings          21.2.0
arviz                         0.15.1
astropy                       5.2.2
astunparse                    1.6.3
attrs                         22.2.0
audioread                     3.0.0
autograd                      1.5
Babel                         2.12.1
backcall                      0.2.0
beautifulsoup4                4.11.2
bleach                        6.0.0
blis                          0.7.9
blosc2                        2.0.0
bokeh                         2.4.3
branca                        0.6.0
CacheControl                  0.12.11
cached-property               1.5.2
cachetools                    5.3.0
cata

In [5]:
!pip install imbalanced-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
# Numpy와 RandomUnderSampler의 로드
import numpy as np
from imblearn.under_sampling import RandomUnderSampler

# 데이터 세트를 설명변수와 목적변수로 분할
X = np.array(bank_df_new.drop('y', axis=1))
Y = np.array(bank_df_new['y'])

# y가 1또는 0인 데이터의 건수를 카운트
print(np.sum(Y == 1), np.sum(Y == 0))

# y가 1인 데이터의 건수에 맞춰서 다운사이징
sampler = RandomUnderSampler(random_state=42)
X, Y = sampler.fit_resample(X, Y)

# y가 1또는 0인 데이터의 건수를 카운트
print(np.sum(Y == 1), np.sum(Y == 0))

820 6113
820 820


**----여기서부터 추가----**

In [7]:
from sklearn.feature_selection import SelectKBest

# 특징량을 5개 선택
selector = SelectKBest(k=5) 
selector.fit(X, Y)
mask = selector.get_support()

# 어느 변수를 선택하였는지 확인
print(bank_df_new.drop('y', axis=1).columns)  # 이거 4. 다시 데이터로 에서 마지막 파트 어느 변수 영향이 잇는지에 함께 붙여서 쓰자.
print(mask)

Index(['age', 'default', 'balance', 'housing', 'loan', 'day', 'duration',
       'campaign', 'pdays', 'previous', 'divorced', 'married', 'single',
       'primary', 'secondary', 'tertiary', 'cellular', 'telephone', 'unknown',
       'apr', 'aug', 'dec', 'feb', 'jan', 'jul', 'jun', 'mar', 'may', 'nov',
       'oct', 'sep', 'non-worker', 'worker', '1Q', '2Q', '3Q', '4Q', 'early',
       'late', 'midle', 'long', 'short', 'one-more', 'zero', 'less', 'more'],
      dtype='object')
[False False False  True False False  True False False False False False
 False False False False False False  True False False False False False
 False False False False False False False False False False False False
 False False False False  True  True False False False False]


In [8]:
# 선택한 특징량의 열만 취득
X_sel5 = selector.transform(X)
print(X_sel5)

[[  1  79   0   0   1]
 [  1  70   1   0   1]
 [  0 108   0   0   1]
 ...
 [  0 357   0   1   0]
 [  0 449   0   1   0]
 [  0 386   0   1   0]]


**----여기까지 추가----**

#### 결정 트리 모델의 작성과 검증

In [21]:
# Scikit-learn의 로드
from sklearn.model_selection import KFold
from sklearn import tree
from sklearn.metrics import accuracy_score

# K-Fold교차 검증
kf = KFold(n_splits=10, shuffle=True)
# 모델의 정밀도를 보존하기 위한 준비
scores = []
# 데이터를 셔플하고, 훈련데이터와 테스트 데이터를 분활
for train_id, test_id in kf.split(X_sel5):
    # 훈련 데이터를 사용한 모델을 작성
    x = X_sel5[train_id]
    y = Y[train_id]
    clf = tree.DecisionTreeClassifier()
    clf.fit(x,y)
    # 테스트 데이터에 모델을 적용
    pred_y = clf.predict(X_sel5[test_id])
    # 모델의 정밀도를 계산하고 보존
    score = accuracy_score(Y[test_id], pred_y)
    scores.append(score)

# 모델의 평균정밀도, 표준편차를 확인
scores = np.array(scores)
print(scores.mean(), scores.std())

0.7189024390243903 0.033112619469232836


In [22]:
# Scikit-learn의 로드
#from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

# 재현률, 적합률의 확인
#print(confusion_matrix(Y[test_id], pred_y))
print(recall_score(Y[test_id], pred_y))
print(precision_score(Y[test_id], pred_y))

0.6463414634146342
0.726027397260274


#### 모델의 정밀도 향상

In [ ]:
# 모델작성 패러메터의 확인
print(clf)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')


In [14]:
# Scikit-learn의 로드
from sklearn.model_selection import GridSearchCV

# 패러메터의 범위설정
params = {
    'criterion': ['entropy'],    
    'max_depth': [2, 4, 6, 8, 10],
    'min_samples_leaf': [10, 20, 30, 40, 50],
}

# 그리드 서치의 실행조건을 설정
clf_gs = GridSearchCV(tree.DecisionTreeClassifier(), params, 
                      cv=KFold(n_splits=10, shuffle=True), scoring='accuracy')

# 그리드 서치 실행
clf_gs.fit(X, Y)

GridSearchCV(cv=KFold(n_splits=10, random_state=None, shuffle=True),
             estimator=DecisionTreeClassifier(),
             param_grid={'criterion': ['entropy'],
                         'max_depth': [2, 4, 6, 8, 10],
                         'min_samples_leaf': [10, 20, 30, 40, 50]},
             scoring='accuracy')

In [13]:
# 가장 높은 정밀도와 패러메터의 조합을 표시
print(clf_gs.best_score_)
print(clf_gs.best_params_)

0.7945121951219513
{'criterion': 'entropy', 'max_depth': 6, 'min_samples_leaf': 20}
